# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [6]:
!pip install -q schedule pytest # установка библиотек, если ещё не

In [ ]:
# Библиотеки, которые могут вам понадобиться
# При необходимости расширяйте список
import time
import requests
import schedule
from bs4 import BeautifulSoup

## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [20]:
def get_book_data(book_url: str) -> dict:
    """
    Извлекает данные о книге с веб-страницы.
    
    Args:
        book_url (str): URL-адрес страницы с информацией о книге
    
    Returns:
        dict: словарь с данными о книге:
            title (str): название книги
            price (str): цена книги
            rating (str): рейтинг книги
            in_stock (str): информация о наличии
            description (str): описани книги
            product_info (dict): дополнительные характеристики из таблицы Product Information
    Raises:
        requests.RequestException: если ошибка при запросе к веб-странице
    """
    
    # НАЧАЛО ВАШЕГО РЕШЕНИЯ    
    result = {}
    session = requests.Session()

    try:
        # устанавливаем соединение с сайтом
        response = session.get(book_url, timeout=5)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, "html.parser")

        # извлекаем информацию и обрабатываем ситуацию, когда какого-либо значения нет

        # извлекаем название книги
        title = soup.find("h1")
        result["title"] = title.text.strip() if title else ""

        # извлекаем цену и обрабатываем неизвестный символ в цене
        price = soup.find(class_="price_color")
        result["price"] = price.text.strip().replace("Â", "") if price else ""

        # извлекаем рейтинг
        rating_elem = soup.find(class_="star-rating")
        if rating_elem:
            classes = rating_elem.get("class")
            # оставляем только класс, соответствующий рейтингу
            rating_class = [cls for cls in classes if cls != "star-rating"]
            result["rating"] = rating_class[0] if rating_class else ""
        else:
            result["rating"] = ""

        # извлекаем информацию о наличию книги на складе
        in_stock = soup.find(class_="instock availability")
        result["in_stock"] = in_stock.text.strip() if in_stock else ""

        # извлекаем описание книги
        desc_div = soup.find("div", id="product_description")
        if desc_div:
            desc_par = desc_div.find_next_sibling("p")
            result["description"] = desc_par.text.strip() if desc_par else ""
        else:
            result["description"] = ""

        # извлекаем дополнительную информацию о книге из таблицы Product Information
        table = soup.find("table", class_="table-striped")
        product_info = {}

        if table:
            # извлекаем ряды
            rows = table.find_all("tr")
            for row in rows:
                # извлекаем "заголовки" и "значения"
                header = row.find("th")
                value = row.find("td")
                if header and value:
                    header_text = header.text.strip()
                    value_text = value.text.strip()
                    # обрабатываем неизвестный символ в ценах
                    if "Price" in header_text or "Tax" in header_text:
                        value_text = value_text.replace("Â", "")
                    product_info[header_text] = value_text

        result["product_info"] = product_info
        return result

    except requests.RequestException as e:
        print(f"Ошибка {e} при запросе к {book_url}")
        return {}
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [21]:
# Используйте для самопроверки
book_url = 'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
get_book_data(book_url)

{'title': 'A Light in the Attic',
 'price': '£51.77',
 'rating': 'Three',
 'in_stock': 'In stock (22 available)',
 'description': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe plac

## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [24]:
def scrape_books(base_url='http://books.toscrape.com/', is_save=False):
    """
    Проходится по всем страницам из каталога на веб-странице, 
    осуществляет парсинг всех страниц с помощью get_book_data().

    Args:
        is_save (bool): если True - сохраняет результат в файл books_data.txt, 
        если False - возвращает список без сохранения

    Returns:
        list: список словарей с данными о книгах
    """
    
    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    all_books_result = []
    page_num = 1
    # создаем сессии для обработки большого кол-ва запросов
    session = requests.Session()

    # цикл для прохода по каталогам с главной страницы
    while True:
        # создаем "конструктор" адреса каталога
        catalogue_url = f"{base_url}catalogue/page-{page_num}.html"

        # устанавливаем соединение с сайтом
        response = session.get(catalogue_url, timeout=5)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, "html.parser")

        # ищем элементы с информацией о книгах
        books_articles = soup.find_all("article", class_="product_pod")
        # если больше нет - конец каталога, выходим
        if not books_articles:
            break

        # обрабатываем каждую книгу на странице
        for book_article in books_articles:
            # ищем ссылку на страницу с информацией о книге
            link_tag = book_article.find("h3").find("a")
            if link_tag and link_tag.get("href"):
                # относительный URl адрес страницы книги
                book_relative_url = link_tag["href"]

                # обработка URL
                if not book_relative_url.startswith("catalogue/"):
                    book_relative_url = "catalogue/" + book_relative_url
                book_url = base_url + book_relative_url

                # получаем данные о книги с помощью get_book_data()
                try:
                    book_data = get_book_data(book_url)
                    if book_data and book_data.get("title"):
                        all_books_result.append(book_data)

                except Exception as e:
                    print(f"Ошибка {e} при обработке книги")
                    continue

        # проверяем наличие перехода на следующую страницу
        # если нет - значит это последняя страница
        next_li = soup.find("li", class_="next")
        if not next_li:
            break

        # увеличиваем номер страницы
        page_num += 1
        # задержка, чтобы не перегрузить сервер
        time.sleep(0.5)

    # сохраняем информацию в файл, если передан флаг is_save=True
    if is_save and all_books_result:
        # открываем файл и записываем
        with open("../artifacts/books_data.txt", "w", encoding="utf-8") as f:
            # заголовок
            f.write(f"Обработано {len(all_books_result)} книг\n")
            # линия отделения заголовка от остального текста
            f.write(f'{"=" * 60}\n\n')

            # обрабатываем каждую книгу
            for i, book in enumerate(all_books_result, 1):
                f.write(f"Книга № {i}\n")
                f.write(f"Название: {book.get('title', ' ')}\n")
                f.write(f"Цена: {book.get('price', ' ')}\n")
                f.write(f"Рейтинг: {book.get('rating', ' ')}\n")
                f.write(f"В наличии: {book.get('in_stock', ' ')}\n")
                f.write(f"Описание: {book.get('description', ' ')}\n")
                # обрабатываем доп. информацию о книге
                product_info = book.get("product_info", {})
                if product_info:
                    f.write(f"Дополнительная информация:\n")
                    for key, value in product_info.items():
                        f.write(f"{key}: {value}\n")

                # добавляем разделитель между книгами
                if i < len(all_books_result):
                    f.write(f'\n{"-" * 40}\n\n')

    return all_books_result
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [25]:
# Проверка работоспособности функции
res = scrape_books(is_save=True) # Допишите ваши аргументы
print(type(res), len(res)) # и проверки

<class 'list'> 1000


## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [ ]:
def scheduler_setup():
    """
    Основная функция для настройки планировщика.

    Настраивает планировщик для ежедневного вызова функции schedule_function() в 19:00.
    Запускает бесконечный цикл для проверки запланированных задач.
    Останавливается при получении KeyboardInterrupt (ручная остановка Ctrl+C).
    """

    def schedule_function():
        """
        Функция для планировщика, которая вызывает scrape_books() ежедневно в 19:00.
    
        Вызывает функцию scrape_books() с параметром is_save=True, которая сохраняет результат в файл.
        """
    
        current_time = time.strftime("%H:%M")
        print(f"Автозапуск функции сбора данных о книгах в {current_time} начат!")
    
        try:
            scrape_books(is_save=True)
            print(f"Выполнение функции завершено в {time.strftime("%H:%M")}!")
        except Exception as e:
            print(f"Ошибка при выполнении функции: {e}")
    
        print("Следующий запуск завтра в 19:00.")

    # удаляем предыдущие "задания"
    schedule.clear()
    
    # настраиваем выполнение каждый день в 19.00
    # для тестирования локально - было установлено 22.05
    schedule.every().day.at("19:00").do(schedule_function)

    print("Планировщик запущен. Ожидайте 19:00.")
    print("Нажмите Ctrl+C или Interrupt the kernel для остановки.")

    try:
        while True:
            # проверяем, есть ли задачи
            schedule.run_pending()
            # ждем 60 сек перед следующей проверкой
            time.sleep(60)
    except KeyboardInterrupt:
        print(
            "Планировщик остановлен вручную.\nРаботоспособность кода проверена успешно!"
        )

scheduler_setup()

Планировщик запущен. Ожидайте 19:00.
Нажмите Ctrl+C или Interrupt the kernel для остановки.
Автозапуск функции сбора данных о книгах в 22:05 начат!
Выполнение функции завершено в 22:14!
Следующий запуск завтра в 19:00.
Планировщик остановлен вручную.
Работоспособность кода проверена успешно!


## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [28]:
# Ячейка для демонстрации работоспособности
# Сам код напишите в отдельном скрипте
! python -m pytest ../tests/test_scraper.py -v

============================= test session starts ==============================
platform darwin -- Python 3.14.0, pytest-8.4.2, pluggy-1.6.0 -- /Users/friemari/Desktop/MIPT/Python/book_scraper/.venv/bin/python
cachedir: .pytest_cache
rootdir: /Users/friemari/Desktop/MIPT/Python/book_scraper
collected 4 items                                                              

../tests/test_scraper.py::test_get_book_data PASSED                      [ 25%]
../tests/test_scraper.py::test_get_books_count PASSED                    [ 50%]
../tests/test_scraper.py::test_book_values PASSED                        [ 75%]
../tests/test_scraper.py::test_gbd_false_url PASSED                      [100%]

======================== 4 passed in 628.51s (0:10:28) =========================


## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```